In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## Load the data and sample a part to have as 'train' data

In [31]:
airports_processed = pd.read_csv("../../2008_processed.csv")
# airports_full = airports_full.drop(["ArrTime", "ActualElapsedTime", "AirTime", "TaxiIn", "Diverted", "CarrierDelay",
#                          "WeatherDelay", "NASDelay", "SecurityDelay", "LateAircraftDelay"], axis=1)
airports = airports_processed.sample(frac=0.9, replace=False, random_state=42)
airports_full_sample = airports_full.sample(frac=0.9, replace=False, random_state=42)

In [14]:
airports

,Unnamed: 0,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,CRSElapsedTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiOut,DepTS,CSRDepTS
1685402,1743151,2008,3,25,2,1149.0,1152,1720,CO,128,N37408,208.0,-16.0,-3.0,RNO,IAH,1530,8.0,2008-03-25 11:49:00,2008-03-25 11:52:00
1240526,1282794,2008,3,16,7,1804.0,1725,1922,XE,3142,N14942,117.0,38.0,39.0,EWR,CMH,462,28.0,2008-03-16 18:04:00,2008-03-16 17:25:00
2113814,2177368,2008,4,25,5,2324.0,2020,2325,MQ,3589,N676AE,125.0,183.0,184.0,DFW,TYS,772,21.0,2008-04-25 23:24:00,2008-04-25 20:20:00
988396,1023743,2008,2,5,2,1759.0,1800,46,NW,1198,N582NW,226.0,2.0,-1.0,LAS,DTW,1750,17.0,2008-02-05 17:59:00,2008-02-05 18:00:00
2104520,2167843,2008,4,26,6,818.0,820,935,MQ,3175,N806AE,75.0,-10.0,-2.0,SJC,LAX,308,12.0,2008-04-26 08:18:00,2008-04-26 08:20:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037571,1075653,2008,2,18,1,1552.0,1535,1825,AA,1181,N3BEAA,230.0,19.0,17.0,PHL,DFW,1302,12.0,2008-02-18 15:52:00,2008-02-18 15:35:00
2258773,2328332,2008,4,6,7,1925.0,1700,1825,B6,1069,N178JB,85.0,154.0,145.0,BOS,JFK,187,18.0,2008-04-06 19:25:00,2008-04-06 17:00:00
385335,397482,2008,1,8,2,1518.0,1510,1720,MQ,3806,N805AE,130.0,16.0,8.0,EVV,DFW,642,11.0,2008-01-08 15:18:00,2008-01-08 15:10:00
93654,94885,2008,1,26,6,1638.0,1650,1811,XE,143,N18557,141.0,-15.0,-12.0,COS,ONT,789,10.0,2008-01-26 16:38:00,2008-01-26 16:50:00


In [ ]:
airports.count()

In [ ]:
for col in airports.columns:
    print(col, len(airports[col].unique())) 

In [ ]:
# Flightnums are used by different carriers to identify their trips. 
airports_full[airports_full['FlightNum'] == 1423].sort_values(by='Month', inplace=False)

## Select factors based on correlations

In [16]:
corr_matrix = airports.corr()
corr_matrix_full = airports_full_sample.corr() # TODO: delete

In [19]:
processed_sort = corr_matrix['ArrDelay'].sort_values()

In [20]:
full_sort = corr_matrix_full['ArrDelay'].sort_values()

In [23]:
compare = pd.DataFrame()
compare['processed'] = processed_sort
compare['full'] = full_sort
compare['diff'] = full_sort - processed_sort
compare

,processed,full,diff
Month,-0.033901,-0.015513,0.018387
Unnamed: 0,-0.027204,NaN,NaN
DayofMonth,-0.024172,0.023977,0.048149
Distance,-0.008208,0.023804,0.032012
CRSElapsedTime,-0.002465,0.019811,0.022276
DayOfWeek,-0.000181,0.017746,0.017927
FlightNum,0.033300,-0.025337,-0.058637
CRSArrTime,0.105064,0.091844,-0.013220
CRSDepTime,0.107687,0.088501,-0.019186
DepTime,0.175870,0.117615,-0.058255


In [ ]:
# Check the scatterplot
sns.scatterplot(data=airports, x= 'DepDelay', y = 'ArrDelay')
plt.show()

In [ ]:
sns.scatterplot(data=airports, x= 'Distance', y = 'ArrDelay')
plt.show()

In [ ]:
airports['ResDelay'] = airports['ArrDelay'] - airports['DepDelay']
sns.scatterplot(data=airports, x= 'Distance', y = 'ResDelay')
plt.show()

## Variables based on categories
For example day of week can be seen as category. Weekends can be more busy. As is shown in data as well by second plot with mean of delay per day.
The errors in the plots are the standard error of the mean, defined as:
$$ \sigma_{\bar{x}} = \frac{\sigma}{\sqrt{n}}$$
In each of the categories there seems to be a difference. 
I have created a new variable called "ResDelay", this is a residual delay after subtracting the departure delay from the arrival delay. It could for example be that one company always departs before planned time and therefore has low arrival delays. Plots are made for different variables.

In [ ]:
def plot_with_error_bars(x, y, data=airports):
    """Function that plots data with the mean of y grouped by the values of x"""
    
    grouped_month = data.groupby(x).agg([np.mean, np.std, 'count'])[y]
    # Standard error of mean is defined as std / sqrt(n) 
    grouped_month['error'] = grouped_month['std'] / np.sqrt(grouped_month['count'])
    
    if type(x) == str: 
        title = y + " by " + x
    else:
        title = y + " by " + " and ".join(x)
    grouped_month.plot(kind = "bar", y = "mean", legend = False,  
              yerr = "error", title = title)
    plt.show()

In [ ]:
sns.boxplot(data=airports, x= 'DayOfWeek', y = 'ArrDelay')
plt.show()
plot_with_error_bars(x='DayOfWeek', y='ArrDelay')
plot_with_error_bars(x='DayOfWeek', y='ResDelay')

In [ ]:
plot_with_error_bars(x='Month', y='ArrDelay')
plot_with_error_bars(x='Month', y='ResDelay')

In [ ]:
plot_with_error_bars(x='UniqueCarrier', y='ArrDelay')
plot_with_error_bars(x='UniqueCarrier', y='ResDelay')

In [ ]:
plot_with_error_bars(x='DayofMonth', y="ArrDelay")
plot_with_error_bars(x='DayofMonth', y="ResDelay")


In [ ]:
airports['DepHour'] = airports["CRSDepTime"]//100

plot_with_error_bars(x='DepHour', y="ArrDelay")
plot_with_error_bars(x='DepHour', y="ResDelay")


In [ ]:
airports['ArrHour'] = airports["CRSArrTime"]//100

plot_with_error_bars(x='ArrHour', y="ArrDelay")
plot_with_error_bars(x='ArrHour', y="ResDelay")


Below are some plots that are not very clear, but there seem to be some difference between origin and destinations. 

In [ ]:
plot_with_error_bars(x='Origin', y="ArrDelay")
plot_with_error_bars(x='Origin', y="ResDelay")

In [ ]:
plot_with_error_bars(x='Dest', y="ArrDelay")
plot_with_error_bars(x='Dest', y="ResDelay")

In [ ]:
plot_with_error_bars(x=['UniqueCarrier', 'DayOfWeek'] , y='ArrDelay')
plot_with_error_bars(x=['UniqueCarrier', 'DayOfWeek'], y='ResDelay')

In [ ]:
sns.scatterplot(data=airports_df, x= 'TaxiOut', y = 'ArrDelay')
plt.show()

In [ ]:
airports_df['ResDelay'] = airports_df['ArrDelay'] - airports_df['DepDelay']
sns.scatterplot(data=airports_df, x= 'TaxiOut', y = 'ResDelay')
plt.show()